# 빅오를 활용한 알고리즘 성능 분석 / 개선

- python의 `timeit`, `cProfile` module 

In [1]:
import timeit 
timeit.timeit('a,b = 42, 101; a = a ^ b; b = a ^ b; a = a ^ b;  a = a ^ b')

0.0648015829938231

In [2]:
import timeit
timeit.timeit('a,b = 42, 101; temp = a; a = b; b = temp')

0.030353207999723963

In [3]:
#반복가능 unpacking
import timeit
timeit.timeit('a, b =42, 101; a,b = b,a')

0.02426275001198519

In [4]:
a,b = 42, 101
a, b = b, a
print(a, b)

101 42


# a, b = b, a는 내부적으로 어떻게 동작하나?
파이썬의 다중 대입(언패킹)은 RHS를 평가해 LHS에 순서대로 바인딩하며, CPython은 두 값 스왑에 최적화된 바이트코드를 사용합니다.
## 예시 파일
[dis 모듈(바이트코드 디스어셈블러) 공식 문서](https://docs.python.org/ko/3/library/dis.html)
## 답변
핵심 요약:
- 언어 차원에서 a, b = b, a는 “RHS 평가 → LHS에 순서대로 할당”이라는 튜플 패킹/언패킹 규칙을 따릅니다.
- CPython 구현은 두 값 스왑에 대해 실제 튜플을 만들지 않고 스택을 회전하는 바이트코드(ROT_TWO)로 최적화합니다. 그래서 빠르고 메모리 효율적입니다.
- timeit으로 비교하면 a, b = b, a가 임시 변수나 XOR 스왑보다 일반적으로 빠르고 가독성도 좋습니다.

실행 흐름(의사 단계):
1) RHS 평가: b와 a를 평가해 두 값을 스택에 올림.
2) 스택 회전: ROT_TWO로 두 스택 값을 교환.
3) LHS 저장: STORE_FAST a, STORE_FAST b로 각 변수에 저장.

실제 바이트코드 확인:
````python
import dis

def swap(a, b):
    a, b = b, a
    return a, b

dis.dis(swap)
# 출력 예(버전에 따라 약간 다름):
#  2           0 LOAD_FAST                1 (b)
#              2 LOAD_FAST                0 (a)
#              4 ROT_TWO
#              6 STORE_FAST               0 (a)
#              8 STORE_FAST               1 (b)
#  3          10 LOAD_FAST                0 (a)
#             12 LOAD_FAST                1 (b)
#             14 BUILD_TUPLE              2
#             16 RETURN_VALUE
````

timeit 비교(스왑 방식 성능):
````python
import timeit

print("XOR:", timeit.timeit('a,b=42,101; a=a^b; b=a^b; a=a^b', number=500_000))
print("temp:", timeit.timeit('a,b=42,101; t=a; a=b; b=t', number=500_000))
print("unpack:", timeit.timeit('a,b=42,101; a,b=b,a', number=500_000))
# 보통 unpack(다중 대입)이 가장 빠르고 간결합니다.
````

왜 XOR 스왑이 부적합한가:
- 파이썬의 int는 가변 길이 객체이며, XOR 스왑은 저수준 정수 레지스터 가정에 유리하지만 파이썬 객체 레벨에서는 오버헤드가 큽니다.
- 가독성/안전성 측면에서도 a, b = b, a가 명확합니다.

추가로, timeit 사용 방식:
- timeit.timeit(stmt, number=N)는 stmt를 N번 반복 실행한 총 소요 시간을 반환합니다.
- 문자열로 전달된 코드 조각은 별도 작은 스코프에서 실행되며, 측정 잡음을 줄이기 위해 최적화되어 있습니다.

### 추가 자료
- [파이썬 데이터 모델: 대입과 언패킹](https://docs.python.org/ko/3/reference/simple_stmts.html#assignment-statements)
- [timeit 모듈 문서](https://docs.python.org/ko/3/library/timeit.html)
- [CPython 바이트코드 명령 목록(dis)](https://docs.python.org/ko/3/library/dis.html#bytecode-instructions)

In [ ]:
import timeit
timeit.timeit('random.randint(1, 100)', 'import random', number=1000000)

0.28861966700060293

In [7]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "import timeit \ntimeit.timeit('a,b = 42, 101; a = a ^ b; b = a ^ b; a = a ^ b;  a = a ^ b')",
  "import timeit\ntimeit.timeit('a,b = 42, 101; temp = a; a = b; b = temp')",
  "#반복가능 unpacking\nimport timeit\ntimeit.timeit('a, b =42, 101; a,b = b,a')",
  'a,b = 42, 101\na, b = b, a\nprint(a, b)',
  "import timeit\ntimeit.timeit('random.randint(1, 100)', 'import random', number=1000000)",
  'global()',
  'globals()'],
 '_oh': {1: 0.0648015829938231,
  2: 0.030353207999723963,
  3: 0.02426275001198519,
  5: 0.28861966700060293},
 '_dh': [PosixPath('/Users/donghun2/workspace/oject_oriented_python/cleancode')],
 'In': ['',
  "import timeit \ntimeit.timeit('a,b = 42, 101; a = a ^ b; b = a ^ b; a = a ^ b;  a = a ^ b')"

# globals 

- [Globals](./globals.md). 


# cProfile 프로파일러

- 작은 코드 조각을 측정하는데는 timeit module이 유용하지만, 전체 함수나 프로그램을 분석하는데는 `cProfile` 모듈이 더 효과적 

In [8]:
import time, cProfile
def addUpNumbers():
    total = 0
    for i in range(1, 1000001):
        total += i
cProfile.run('addUpNumbers()')

         261 function calls (259 primitive calls) in 0.088 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    0.005    0.005 2682080545.py:2(addUpNumbers)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
      2/1    0.013    0.007    0.058    0.058 <string>:1(<module>)
        1    0.000    0.000    0.074    0.074 asyncio.py:206(_handle_events)
        1    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        1    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    0.000    0.000    0.000 base_events.py:1947(_add_callback)
        1    0.000    0.000    0.000    0.000 base_events.py:772(time)
        1    0.000    0.000    0.000    0.000 contextlib.py:108(__init__)
        1    0.000    0.000    0.000    0.000 contextlib.py:136(__enter__)
        1    0.000    0.000    0.000    0.000 contextlib.p

# example code 

- https://nostarch.com/download/CrackingCodesFiles.zip

In [11]:
import sys
from pathlib import Path

# 노트북 실행 위치를 모듈 검색 경로에 추가
sys.path.append(str(Path().resolve()))
import rsaCipher  # 또는 rsaChiper
import cProfile, rsaCipher
cProfile.run("rsaCipher.encryptAndWriteToFile('encrypted_file.txt', 'al_sweigart_pubkey.txt', 'abc'*100000)")

         12423 function calls (12412 primitive calls) in 39.436 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:189(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:263(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:312(__init__)
        1    0.000    0.000    0.000    0.000 <frozen codecs>:322(decode)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    8.584    8.584 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 asyncio.py:231(add_callback)
        3    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        3    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        4    0.000    0.000    0.000    0.000 ba